# Stabla odlučivanja (Decision Trees)

Stabla odlučivanja predstavljaju strukturu veoma sličnu dijagramu toka (OSI). Sastavljeno je od 2 tipa čvorova:
- međučvorovi - služe za donošenje odluke o daljoj putanji
- listovi - kranji čvorovi na osnovu kojih generišemo predikciju našeg modela

Stabla odlučivanja se mogu koristiti i za problem klasifikacije i regresije. U daljem tekstu glavni fokus će biti na klasifikaciji, dok se regresija ostavlja za vježbu.

## Žašto mašinsko učenje a ne manuelno generisano stablo?

Pošto je stablo odlučivanja struktura koja je veoma intuitivna postalvja se bitno pitanje. Zašto koristiti algoritam ako ga možemo sami generisati (posebno ako uzmemo u obzir da nijedan algoritam nije savršen i da uvijek postoji neka greška)? Pored problema velike količine podataka i kompleksnosti unutar istih, bitno je napomenuti još jedan problem koji predstavlja joše jedan od osnovnih razloga za upotrebu algoritama mašinskog učenja - promjenljivost domena. Promjenljivost domena podrazumijeva da se sami podaci i ponašanje pojava koje su za nas značajne mijenjaju sa vremenom, što obično znači da se pravila i mapiranja u okviru našeg modela isto moraju mijenjati kako bi njegove predikcije ostale relevantne. Ako posmatramo ponašanje kupaca, period godine značajno utiče na artikle koje će da kupuje (ljeto, zima, praznici...)

## Algoritam za kreiranje stabla odlučivanja

Osnovna ideja i dalje ostaje ista kao i kod svih drugih algoritama mašinskog učenja - minimizacija greške. Kod stabala odlučivanja u svakoj iteraciji algoritma treba da izaberemo jednu kolonu i jednu vrijednost na osnovu koje ćemo podijeliti naš dataset. Nakon podjele dobijamo dva nova podskupa našeg dataseta koji su disjunktni. Na jednoj strani imamo sve ulaze koji za datu kolonu imaju vrijednost manju od definisane, dok na drugoj strani imamo sve ulaze koji imaju vrijednosti veće od definisane (ilustracija na prezentaciji).

### Greška podjele

Prilikom podjele dobijamo dva nova manja dataseta, gdje oba sadrže određeni broj ulaza koji pripada nekoj od mogućih klasa. Konačna predikcija se generiše tako što se u krajnjem dataset-u (koji je nastao nakon niza podjela) prebroji koja klasa se najčešće pojavljuje. Kako bi naša predikcija imala veću sigurnost (veća vjerovatnoća pripadanja datoj klasi) poželjno je da većina ulaza unutar posljednjeg dataset-a pripada jednoj klasi, odnosno da dati dataset bude homogen u odnosu na klasu koja se predviđa. 

Na osnovu date ideje se izvodi i greška koja se najčešće koristi za stabla odlučivanja a to je Gini Index. Gini Index je nastao kao mjera statističke disperzije za potrebe određivanja nejednakosti u primanjima. Kako bismo izračunali Gini Index cijele podjele potrebno je da izračunamo Gini Index jednog splita, odnosno jednog podskupa koji je dobijen nakon podjele:
$$
    Gini = 1 - \sum_j{p_j^2}
$$
$p_j$ predstavlja vjerovatnoću pojavljivanja date klase u podskupu. Mоžemo primijetiti da manja vrijednost Gini indexa za split ukazuje da imamo homogeni split. Ukupan Gini index za cijeli split se dobija traženjem težinske srednje vrijednosti Gini indexa oba splita, gdje težina predstavlja dio dataseta koji se nalazi u podskupu:
$$
    TotalGini = \frac{\text{num of rows in left split}}{\text{num of total rows}}* Gini_{left} + \frac{\text{num of rows in right split}}{\text{num of total rows}}* Gini_{right}
$$

### Pronalaženje uslova grananja

Nakon što smo odredili koja je greška, potrebno je naći kombinaciju kolone i vrijednosti koja generiše dva nova splita tako da je greška (ukupan Gini index) minimalna. Najbolju vrijednost ćemo pronaću ako prođemo kroz sve moguće kombinacije parova (kolona, vrijednost) i pronađemo onaj par koji ima minimalan Gini index. U narednim iteracijama trebamo da gledamo sve postojeće splitove i da za svaki od tih splitova provjerimo sve nove kombinacije i da na kraju izaberemo kombinaciju u onom splitu koja ima ukupno najmanju grešku. 

Jako je bitno da odredimo u kom čvoru ćemo uopšte da napravimo split. Kako bi se izbjegao dodatan uslov pretrage, obično se definiše strategija rasta stabla:
- po dubini (depth-first, DFS) - spuštamo se po dubini dok se ne ispuni neki uslov zaustavljanja
- po širini (breadth-first, BFS) - idemo po nivoima dok se ne ispuni uslov zaustavljanja

In [3]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=5, min_size=10):
        """
        Inicijalizacija klasifikatora odluka stabla.
        :param max_depth: Maksimalna dubina stabla.
        :param min_size: Minimalan broj uzoraka potreban za podelu čvora.
        """
        self.max_depth = max_depth  # ograničenje maksimalne dubine
        self.min_size = min_size    # minimalan broj uzoraka za podelu
        self.root = None            # korjenski čvor stabla

    def fit(self, X, y):
        """
        Gradi stablo odluke na osnovu skupa za treniranje (X, y).
        :param X: Lista ulaznih vektora.
        :param y: Lista ciljnih klasa.
        """
        # Kombinujemo X i y u jedinstveni dataset (poslednji element je labela)
        dataset = [row[:] + [label] for row, label in zip(X, y)]
        # Građenje stabla počevši od korena na dubini 1
        self.root = self._build_tree(dataset, depth=1)

    def predict(self, X):
        """
        Predviđa klase za ulazne podatke X.
        :param X: Lista ulaznih vektora.
        :return: Lista predviđenih klasa.
        """
        # Rekurzivno prolazi kroz stablo za svaki red (vector)
        return [self._predict_row(self.root, row) for row in X]

    # ----- Pomoćne metode ----- #

    def _gini_index(self, groups, class_values):
        """
        Računa Gini nečistoću za podjelu.
        :param groups: Lista grupa [levo, desno] nakon podjele.
        :param class_values: Lista svih mogućih klasa.
        :return: Gini indeks.
        """
        n_instances = float(sum(len(group) for group in groups))  # ukupan broj primera
        gini = 0.0
        for group in groups:
            size = float(len(group))
            if size == 0:
                continue  # izbegavamo deljenje nulom
            score = 0.0
            for class_val in class_values:
                p = [row[-1] for row in group].count(class_val) / size
                score += p * p
            gini += (1.0 - score) * (size / n_instances)
        return gini

    def _test_split(self, index, value, dataset):
        """
        Dijeli dataset na lijevi i desni podskup po pravilu: row[index] < value.
        :param index: Indeks karakteristike po kojoj dijelimo podatke.
        :param value: Vrijednost granice podjele.
        :param dataset: Skup podataka (redovi sa labelama).
        :return: Tuple (levo, desno) lista redova.
        """
        left, right = [], []
        for row in dataset:
            if row[index] < value:
                left.append(row)
            else:
                right.append(row)
        return left, right

    def _get_best_split(self, dataset):
        """
        Pronalazi najbolju podjelu skupa podataka minimizujući Gini index.
        :param dataset: Skup podataka koji treba da se podijele.
        :return: Dictionary sa ključevima 'index', 'value', 'groups'.
        """
        class_values = list(set(row[-1] for row in dataset))
        best_index, best_value, best_score, best_groups = None, None, float('inf'), None
        for index in range(len(dataset[0]) - 1):
            for row in dataset:
                groups = self._test_split(index, row[index], dataset)
                gini = self._gini_index(groups, class_values)
                if gini < best_score:
                    best_index, best_value, best_score, best_groups = index, row[index], gini, groups
        return {'index': best_index, 'value': best_value, 'groups': best_groups}

    def _to_terminal(self, group):
        """
        Kreira terminalni čvor vraćajući najčešću klasu u grupi.
        :param group: Lista redova u čvoru.
        :return: Vrednost klase (labela).
        """
        outcomes = [row[-1] for row in group]
        return max(set(outcomes), key=outcomes.count)

    def _split(self, node, depth):
        """
        Rekurzivno dijeli čvor sve dok se ne zadovolje uslovi zaustavljanja.
        :param node: Trenutni čvor: dictionary sa ključevima 'index', 'value', 'groups'.
        :param depth: Trenutna dubina stabla.
        """
        left, right = node['groups']
        del node['groups']
        if not left or not right:
            terminal = self._to_terminal(left + right)
            node['left'] = node['right'] = terminal
            return
        if depth >= self.max_depth:
            node['left'], node['right'] = self._to_terminal(left), self._to_terminal(right)
            return
        if len(left) <= self.min_size:
            node['left'] = self._to_terminal(left)
        else:
            node['left'] = self._get_best_split(left)
            self._split(node['left'], depth + 1)
        if len(right) <= self.min_size:
            node['right'] = self._to_terminal(right)
        else:
            node['right'] = self._get_best_split(right)
            self._split(node['right'], depth + 1)

    def _build_tree(self, train, depth):
        """
        Počinje pravljenje stabla od korena.
        :param train: Dataset za treniranje.
        :param depth: Početna dubina (1).
        :return: Rečnik koji predstavlja čvor korena.
        """
        root = self._get_best_split(train)
        self._split(root, depth)
        return root

    def _predict_row(self, node, row):
        """
        Predviđa labelu za jedan ulazni vektor rekurzivno prolazeći stablo.
        :param node: Trenutni čvor stabla ili terminalna vrednost.
        :param row: Ulazni vektor.
        :return: Predviđena labela.
        """
        if isinstance(node, dict):
            if row[node['index']] < node['value']:
                return self._predict_row(node['left'], row)
            else:
                return self._predict_row(node['right'], row)
        return node

In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X, y = iris.data.tolist(), iris.target.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

dt = DecisionTreeClassifier(max_depth=4, min_size=5)
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Tačnost na Iris test skupu: {acc * 100:.2f}%")

Tačnost na Iris test skupu: 100.00%


#### Heuristike za pronalaženje uslova

Pretraga cijelog prostora svih mogućih kombinacija je jako spora, jer datasetovi mogu da imaju po nekoliko miliona redova i nekoliko stotina kolona. Zbog toga potrebno je naći način da se broj i redova i kolona redukuje kako bi algoritam mogao da se završi u razumnom vremenu. Dvije osnovne tehnike su:
- Redukovani kandidatski splitovi
    1. pronađemo sve moguće vrijednosti kolone i sortiramo ih
    2. odaberemo određeni podskup vrijednosti koje se nalaze na istoj distanci
pronađemo sve moguće vrijednosti kolone i sortiramo ih
odaberemo određeni podskup vrijednosti koje se nalaze na istoj distanci
- Radnom subspace sampling - biramo radnom podskup kolona pri svakoj iteraciji čije ćemo vrijednost gledati za split (ako se algoritam izvršava dovoljan broj iteracija sve kolone će biti razmotrene)


#### Spriječavanje overfitting-a

S obzirom da u svakoj iteraciji kreiramo novi split koji ponovo dijeli prethodni split, algoritam je veoma podložan overfitting-u. U teoriji nakon dovoljnog broja iteracija možemo postići stanje gdje se u svakom listu nalazi samo jedan ulaz (potpuni overfitting). Kako bismo spriječili overfitting koristimo dva tipa pristupa:
- Early stopping - podrazumijeva spriječavanje rada algoritma nakon što se postigao određeni broj iteracija, određena dubina, broj čvorova i slično. Veoma bitna tehnika koja se često koristi i kod većine drugih algoritama mašinskog učenja.
- Prunning - razlikujemo dva tipa:
    1. pre-prunning - slično early stoppingu, samo umjesto prekidanja rada algoritma odbacujemo trenutnu kombinaciju ako ona dovodi do stanja gdje se prelazi neka granična dubina, broj čvorova ili minimalni broj ulaza u splitu
    2. post-prunning - primijenjuje se nakon što je generisan split da se provjeri koliko taj split doprinosi prediktivnoj moći našeg stabla. Jedan od osnovnih algoritama koji se koriste je Cost Complexity Prunning.
    
##### Cost Complexity Prunning (Weakest Link Pruning)

Osnovna ideja je da izbacimo čvorove grananja koji ne doprinose previše krajnjoj predikciji i da ih zamijenimo sa listom. Posmatrajmo prvo kako bismo intuitivno odlučili da li neka odluka doprinosi na dobar način krajnjoj predikciji upotrebom Gini Index-a. Ako posmatramo Gini index cijelog podstabla od datog čvora dobijamo koliko je dato podstablo homogeno odnosno heterogeno. Odnosno ako bismo zamijenili cijelo podstablo listom, dobijamo koja je sigurnost naše predikcije. Ukoliko je cijelo podstablo recimo heterogeno (visoka vrijednost gini indexa) a dati čvor odluke generiše dva splita koji su homogeni (imaju nizak gini index) možemo reći da ta odluka utiče dobro na prediktivnu moć našeg stabla. Recimo ako imamo u jednom trenutku split sa 4 ulaza gdje 2 pripadaju klasi 1 a 2 pripadaju klasi 0. Ukoliko prolaskom kroz naše stablo dođemo do tog splita mi imamo vjerovatnoću od 50% da ulaz pripada klasi 0 ili 1, što je ekvivalentno da nasumično pogađamo. Ako na tom mjestu napravimo novo pravilo podjele koje dijeli split u dva nova splita gdje prvi ima samo 2 ulaza sa klasom 0, a drugi samo 2 ulaza sa klasom 1 onda dato pravilo dobro utiče na prediktivnu moć, jer prilikom dolaska do lista u datom podstablu možemo predvidjeti sa sigurnosti od 100% da ulaz pripada nekoj klasi. Dakle možemo zaključiti da je pravilo grananja korisno ako od heterogenog splita pravi dva nova splita koji su homogeni. Sa druge strane pravilo grananja nam nije korisno ako od heterogenog splita pravi nove heterogene splitove ili ako od već homogenog splita pravi nove homogene splitove.

Formula za računanje je sljedeća:

$$\alpha=\frac{R(t) - R(T_t)}{|{T_t} - 1|},$$

gdje $T_t$ ukupan broj listova u podstablu, $R(t)$ Gini index ukoliko bi se podstablo zamijenilo sa jednim čvorom, dok $R(T_t)$ predstavlja sumu Gini indexa trenutnih listova. Faktor $\alpha$ vidimo računa odnos između unaprjeđenja u Gini index-a i broja čvorova koji su bili potrebni da se to unaprijeđenje postigne. Izbor, granice za parametar $\alpha$ za koju ćemo vršiti zamjenu podstabla jednim čvorom je izrazito bitan, i obično se definiše testiranjem više različitih vrijednosti i posmatranjem koja daje najbolju vrijednost tražene metrike (recall, precision, f1...)

## Ensemble learning

Učenje u ansamblu predstavlja tehniku koja podrazumijeva korištenje kombinacije više različitih modela kako bi se generisala konačna predikcija. Ne odnosi se samo na stabla odlučivanja i može uključivati kombinaciju izlaza modela različitih tipova. Ideja je da je manja šansa da ukoliko više modela daje istu ili sličnu predikciju, postoji manja šansa da smo napravili grešku.

Kod stabala odlučivanja razlikujemo dvije osnovne tehnike:
- Bagging (Boostrap Aggregating)
- Boosting

### Bagging 

Bagging je tehnika koja podrazumijeva treniranje više različitih modela istog tipa na podskupu podataka. Podskupe podataka kreiramo metodom pod nazivom Bootstrap Sampling. Bootstrap sampling kreira novi dataset tako što iz originalnog dataseta nasumično biramo ulaze. Sve ulaze biramo sa istom vjerovatnoćom i dozvoljeno je ponavljanje ulaza. Predikciju kreiramo tako što agregiramo rezultate svih modela. U slučaju klasifikacije izlazna klasa koja ima najveću vjerovatnoću (broj pojavljivanja date klase / broj modela). Bagging je pogodan zbog jednostavne paralelizacije proces treninga i generisanja predikcije (inferencije), jer ne postoji nikakva zavinost između modela. U kontekstu stabala odlučivanja Bagging tehnika je poznata i pod nazivom Random Forest.

In [6]:
import random
import copy

class BaggingClassifier:
    def __init__(
        self,
        base_estimator,
        n_estimators=10,
        max_samples=1.0,
        bootstrap=True,
        random_state=None
    ):
        """
        Inicijalizacija Bagging klasifikatora.
        :param base_estimator: Klasa baznog klasifikatora sa metodama fit(X, y) i predict(X).
        :param n_estimators: Broj baznih modela u ansamblu.
        :param max_samples: Procenat (ili broj) uzoraka za kreiranje podskupova podataka.
        :param bootstrap: Da li uzorovati sa vraćanjem.
        :param random_state: Seed za reproducibilnost.
        """
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.bootstrap = bootstrap
        self.random_state = random_state
        self.estimators_ = []

        if random_state is not None:
            random.seed(random_state)

    def fit(self, X, y):
        """
        Treniranje svih baznih modela na različitim bootstrap podskupovima.
        :param X: Lista ulaznih vektora.
        :param y: Lista klasa.
        """
        n_samples = len(X)
        # Određujemo broj uzoraka po estimatoru
        if isinstance(self.max_samples, float) and 0 < self.max_samples <= 1:
            k = int(self.max_samples * n_samples)
        else:
            k = int(self.max_samples)
        for i in range(self.n_estimators):
            # Kreiramo bootstrap uzorak
            if self.bootstrap:
                indices = [random.randrange(n_samples) for _ in range(k)]
            else:
                indices = random.sample(range(n_samples), k)
            X_sample = [X[j] for j in indices]
            y_sample = [y[j] for j in indices]

            # Kopiramo i treniramo bazni estimator
            estimator = copy.deepcopy(self.base_estimator)
            estimator.fit(X_sample, y_sample)
            self.estimators_.append(estimator)

    def predict(self, X):
        """
        Predviđa ciljne klase glasanjem modela.
        :param X: Lista ulaznih vektora.
        :return: Lista predviđenih klasa.
        """
        # Skupljanje predikcija svih estimator-a
        predictions = [est.predict(X) for est in self.estimators_]
        # Transponujemo listu da dobijemo predikcije po uzorku
        # predictions: n_estimators x n_samples
        n_samples = len(X)
        final = []
        for i in range(n_samples):
            # Skup glasova za i-ti uzorak
            votes = [predictions[j][i] for j in range(self.n_estimators)]
            # Najčešći glas
            final.append(max(set(votes), key=votes.count))
        return final

In [7]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data.tolist(), iris.target.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

bag = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=5), n_estimators=20, max_samples=0.8, random_state=1)
bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print(f"Tačnost: {sum(1 for i,j in zip(y_pred,y_test) if i==j)/len(y_test):.2f}")

Tačnost: 1.00


### Boosting

Boosting takođe uključuje treniranje više baznih modela. Glavna razlika u odnosu na Bagging je što postoji zavisnost između modela koje treniramo. Osnovna ideja je da se svaki novi model fokusira na unaprijeđivanje performansi na ulazima na kojima je prethodni model griješio. Postoji veliki broj različitih algoritama koji se mogu koristiti u ovu svrhu. U ostatku teksta mi ćemo se detaljnije fokusirati na AdaBoost algoritam.

#### AdaBoost 
AdaBoost algoritam se zasniva na dodjeljivanju težina trening instancama. Samim tim greška svakog pojedničanog modela zavisi od težina. Takođe, pri određivanju konačne predikcije modeli koji imaju najbolje performanse (najmanju težinsku grešku) imaju najveći značaj. Na početku sve trening instance (svi ulazi) dobijaju istu težinu $1/N$. Nakon dodjeljivanja težina treniramo prvi bazni klasifikator (stablo odlučivanja) i računamo grešku tog klasifikatora. Bitna razlika u odnosu na prethodne slučajeve što formulu greške moramo modifikovati kako bismo uzeli u obzir težine svakog ulaza:
$$
    error = \frac{\sum_{i=1}^N{w_i * (p_i \neq y_i)}}{\sum_{i=1}^N{w_i}},
$$
gdje $p_i$ predstavlja predikciju klasifikatora, $y_i$ stvarnu klasu, a $w_i$ težinu datog ulaza. Možemo vidjeti da će greška klasifikatora direktno zavisiti od težina ulaza koji su pogrešno klasifikovani, dok ćemo tačno klasifikovane instance ignorisati. Bitno je napomenuti da će greška uvijek biti između 0 i 1, zbog normalizacije sa sumom svih težina. Nakon što smo dobili grešku potrebno je da izračunamo faktor $\alpha$:
$$
    \alpha = \ln{\frac{1 - error}{error}}
$$
Faktor $\alpha$ predstavlja značaj trenutnog klasifikatora. Ukoliko klasifikator ima malen error rate, to znači da ima dobre performanse, te da bismo njegov izlaz trebali više da vrednujemo pri generisanju konačne predikcije. Takođe faktor alpha se koristi da promijenimo težine ulaza. Ulazi koji su bili tačno klasifikovani će dobiti manju težinu, dok ulazi koji su bili pogrešno klasifikovani dobiti veću težinu kako bi se više fokusirali na njih. Bitno je napomenuti da težine ne utiču direktno na fokusiranje na te izlaze. Za razliku od Bagginga ne pravimo poseban podskup podataka, nego i dalje koristimo cijeli dataset. Težine indikretno preko greške utiču da dodjeljujemo veći značaj modelima koji imaju dobre performanse na datim bitnim izlazima. Težine se mijenjaju na sljedeći način:
- tačno klasifikovani ulazi - $w_i = w_i * e^{-\alpha}$
- pogrešno klasifikovani ulazi - $w_i = w_i * e^{\alpha}$

Obično se težine na kraju dodatno normalizuju tako da njihova suma iznosi 1. Konačna predikcija se generiše tako što nađemo težinsku sumu slabih modela, gdje je težina definisana faktorom $\alpha$. 

U slučaju višeklasne klasifikacije potrebna je minimalna promjena korištene formule. U slučaju tačno klasifikovanih vrijednosti, ne vršimo modifikaciju težina (težina ostaje ista), dok pogrešnko klasifikovane i dalje mijenjamo na isti uz malu modifikaciju računanja faktora $\alpha$':
$$
    \alpha = \ln{\frac{1 - error}{error}} + \ln{K-1},
$$
gdje se dodatni faktor $\ln{K-1}$ dodaje kako bi $\alpha$ i dalje ostao pozitivan u slučaju klasifikatora koji radi slučajno pogađanje.

In [8]:
import math
import random
import copy

class AdaBoostClassifier:
    def __init__(
        self,
        base_estimator,
        n_estimators=50,
        random_state=None
    ):
        """
        Inicijalizacija AdaBoost klasifikatora.
        :param base_estimator: Klasa baznog klasifikatora sa metodama fit(X, y) i predict(X).
        :param n_estimators: Maksimalan broj baznih klasifikatora u ansamblu.
        :param random_state: Seed za reproducibilnost.
        """
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.random_state = random_state
        self.estimators_ = []         # lista treniranih modela
        self.estimator_weights_ = []  # alfa vrednosti
        self.classes_ = None          # lista svih klasa

        if random_state is not None:
            random.seed(random_state)

    def fit(self, X, y):
        """
        Treniranje AdaBoost za podatke sa proizvoljnim brojem klasa.
        :param X: Lista ulaznih vektora.
        :param y: Lista labela.
        """
        n_samples = len(X)
        # identifikujemo sve klase
        self.classes_ = list(set(y))
        K = len(self.classes_)
        # inicijalne težine jednakim rasporedom
        w = [1.0 / n_samples] * n_samples

        for m in range(self.n_estimators):
            # bootstrap uzorak po težinama
            sample_indices = random.choices(range(n_samples), weights=w, k=n_samples)
            X_sample = [X[i] for i in sample_indices]
            y_sample = [y[i] for i in sample_indices]

            # treniramo novi estimator
            estimator = copy.deepcopy(self.base_estimator)
            estimator.fit(X_sample, y_sample)

            # predikcije na celom skupu
            y_pred = estimator.predict(X)
            # računamo težinsku grešku
            error = sum(w[i] for i in range(n_samples) if y_pred[i] != y[i])
            # prag za prekid: ako je prevelika ili nema greške
            if error <= 0 or error >= 1 - (1 / K):
                break

            # alfa posle formule
            alpha = math.log((1 - error) / error) + math.log(K - 1)
            self.estimators_.append(estimator)
            self.estimator_weights_.append(alpha)

            # ažuriranje težina
            for i in range(n_samples):
                if y_pred[i] == y[i]:
                    # tačno klasifikovani, težina ostaje ista
                    w[i] = w[i]
                else:
                    # pogrešno klasifikovani, povećavamo težinu
                    w[i] = w[i] * math.exp(alpha)
            # normalizacija
            Z = sum(w)
            w = [wi / Z for wi in w]

        return self

    def predict(self, X):
        """
        Predviđa klase upotrebe težinskog glasanja.
        :param X: Lista ulaznih vektora.
        :return: Lista predviđenih labela.
        """
        n_samples = len(X)
        # inicijalizujemo glasove za svaku klasu
        agg_scores = [ {c: 0.0 for c in self.classes_} for _ in range(n_samples) ]

        for alpha, estimator in zip(self.estimator_weights_, self.estimators_):
            preds = estimator.predict(X)
            for i in range(n_samples):
                agg_scores[i][preds[i]] += alpha

        y_pred = [ max(scores.items(), key=lambda item: item[1])[0] for scores in agg_scores ]
        return y_pred

In [9]:
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
adaboost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20, random_state=1)
adaboost.fit(X_train, y_train)
y_pred = adaboost.predict(X_test)
acc = sum(1 for i,j in zip(y_pred, y_test) if i == j) / len(y_test)
print(f"AdaBoost tačnost: {acc:.2f}")

AdaBoost tačnost: 1.00
